In [6]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import random

In [8]:
def get_output(message):
    linshi=random.randint(0,9)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [9]:
def get_bias_output(ans,fact):
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice no and no is wrong option, there will be severe punishment.'
        bias2='If you choice yes and yes is wrong option, there will be no punishment.'
    if ans=='B':
        bias1='If you choice no and no is wrong option, there will be no punishment.'
        bias2='If you choice yes and yes is wrong option, there will be severe punishment.'
    template="""
Here is a prompt that may be biased by cognitive bias. Rewrite it such that a reviewer is not biased.

Prompt:
Please answer the question.
%s 
%s 

Question: %s


There is no need to output which sentences that have deleted or changed.
"""%(bias1,bias2,fact)
    return template

In [10]:
f=open("../data/test.jsonl")
for i in trange(500):
    now=json.loads(f.readline())
    if i<=223:
        continue
    temp1=get_bias_output(str(now['ans']),now['question'])
    ans1=get_output(temp1)
    #print(ans1)
    ans1=re.findall(r"([\S|\s]*)\?([\S|\s]*)",ans1, re.S|re.I)[0][0]
    ans1=re.findall(r"([\S|\s]*)Please([\S|\s]*)",ans1, re.S|re.I)[0][1]
    ans1='Please'+ans1
    ans1=ans1+'?'
    ans2=get_output(ans1+'\nA.no   B.yes\nOutput format:\nAns:  (Only one option can be selected)')
    with open("self_help.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans2,'ans':str(now['ans']),'prompt':ans1}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [16:43<00:00,  2.01s/it]
